Make sure the tf model and test_timestacks folder are in your working directory. Then simply run the code and check the results.

In [ ]:
# import libraries
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import shutil
import random
import csv
from scipy.stats import norm

tf_model_version = "example" # name of tf model folder
IMG_SIZE = 256
IMG_SIZE_WID = 512

# https://stackoverflow.com/questions/22579434/python-finding-the-intersection-point-of-two-gaussian-curves
def solve(m1,m2,std1,std2):
    a = 1/(2*std1**2) - 1/(2*std2**2)
    b = m2/(std2**2) - m1/(std1**2)
    c = m1**2 /(2*std1**2) - m2**2 / (2*std2**2) - np.log(std2/std1)
    return np.roots([a,b,c])

# function used to evaluate how good the model is
def compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir):
    
    folder_name = "test_timestacks/"
    
    image_range = max_uprush - min_uprush
    
    timestack = cv2.imread(folder_name + timestack_name)
    timestack_focus = timestack[:, min_uprush:max_uprush]
    timestack_labelled = cv2.imread(folder_name + timestack_name + "_output.png")
    timestack_annotated = timestack_labelled.copy()
    mask_annotated = cv2.imread(folder_name + timestack_name + "_output.png", cv2.IMREAD_GRAYSCALE)
    
    timestack_count = 0

    tf_images = []
    if tf_model_version == "michaelThompson_imgSize_64":
        # this way of reading/saving models worked for an older version of tensorflow 2.x
        tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")
    else:
        tf_model_xc = tf.keras.models.load_model(tf_model_version + "_model")

    vertical_coordinates = []
    horizontal_coordinates = []
    full_height, full_width, _ = timestack_focus.shape
    snap_width = full_width
    snap_height = int(full_width*(IMG_SIZE/IMG_SIZE_WID)) # make sure this matches CNN model input ratio
    max_snap_start = full_height - snap_height
    snap_start = 0
    snap_counter = 0
    # split the image up into square sections - overlap the last section
    while snap_start < max_snap_start:

        # get square image
        snapshot = timestack_focus[snap_start:(snap_start + snap_height)]
        
        # prepare as tf input
        snapshot_resize = cv2.resize(snapshot, (IMG_SIZE, IMG_SIZE_WID))
        snapshot_resize_nm = np.array(snapshot_resize) / 255
        snapshot_input = snapshot_resize_nm.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)
        
        
        ####### Prepare image correctly for model

        # put into model
        mask_test = tf_model_xc.predict([snapshot_input])[0]
        mask_test = cv2.resize(mask_test, (snap_width, snap_height))
        # get rid of second channel as it is simply 1 - first channel
        mask_test = mask_test[:, :, 0]

        # add mask to mask image
        mask_annotated[snap_start:(snap_start + snap_height), min_uprush:max_uprush] = mask_test * 255
        
        # apply light and dark gaussian interesection method to extract shoreline points
        # use something similar to Otsu's method for each row to draw a boundary line
        for j in range(0, len(mask_test), 1):
            pixel_row = mask_test[j]

            light_pixels = []
            dark_pixels = []
            for k in range(0, len(pixel_row), 1):
                if pixel_row[k] >= 0.5:
                    light_pixels.extend([k]*int(pixel_row[k]*255))
                else:
                    dark_pixels.extend([k]*int((0.5 - pixel_row[k])*255))

            # find mean and variance of pixels
            light_mean, light_std = norm.fit(light_pixels)
            dark_mean, dark_std = norm.fit(dark_pixels)

            try:
                # get the interect of both gaussians
                intersect_i = solve(light_mean, dark_mean, light_std, dark_std)

                # if intersect has more than 1 point, choose the point closest to the middle of the two means
                if len(intersect_i) == 2:
                    mean_middle = (light_mean + dark_mean)/2
                    intersect = min(intersect_i, key=lambda x:abs(x-mean_middle))
                # if 1 point thats great!
                elif len(intersect_i) == 1:
                    intersect = intersect_i
                # if 0 or 3 or more points, simply choose middle of two means
                else:
                    intersect = (light_mean + dark_mean)/2
            except:
                # just use middle of means
                if np.isnan(light_mean) or np.isnan(dark_mean):
                    intersect = 0
                else:
                    intersect = (light_mean + dark_mean)/2
            
            # annotate over timestack
            cv2.circle(timestack_annotated, (min_uprush + int(intersect), snap_start + j), 1, (255, 0, 0))
            
            # save vertical and horizontal coordinates for comparison with labels.
            horizontal_coordinates.append(snap_start + j)
            vertical_coordinates.append(min_uprush + int(intersect))
        
        timestack_count = timestack_count + 1
        snap_start = snap_start + int(snap_height)
        snap_counter = snap_counter + 1
    
    snap_end = snap_counter*snap_height
    # after while loop is finished, do one final snapshot that overlaps with last snapshot till the end.
    # get square image
    snapshot = timestack_focus[(full_height - snap_height):full_height]

    # prepare as tf input
    snapshot_resize = cv2.resize(snapshot, (IMG_SIZE, IMG_SIZE_WID))
    snapshot_resize_nm = np.array(snapshot_resize) / 255
    snapshot_input = snapshot_resize_nm.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)

    # put into model
    mask_test = tf_model_xc.predict([snapshot_input])[0]
    mask_test = cv2.resize(mask_test, (snap_width, snap_height))
    # get rid of second channel as it is simply 1 - first channel
    mask_test = mask_test[:, :, 0]
    
    # add mask to mask image
    mask_annotated[(full_height - snap_height):full_height, min_uprush:max_uprush] = mask_test * 255
    
    # apply light and dark gaussian interesection method to extract shoreline points
    # use something similar to otsu's method for each row to draw a boundary line
    for j in range(0, len(mask_test), 1):
        pixel_row = mask_test[j]

        light_pixels = []
        dark_pixels = []
        for k in range(0, len(pixel_row), 1):
            if pixel_row[k] >= 0.5:
                light_pixels.extend([k]*int(pixel_row[k]*255))
            else:
                dark_pixels.extend([k]*int((0.5 - pixel_row[k])*255))

        # find mean and variance of pixels
        light_mean, light_std = norm.fit(light_pixels)
        dark_mean, dark_std = norm.fit(dark_pixels)

        try:
            # get the interect of both gaussians
            intersect_i = solve(light_mean, dark_mean, light_std, dark_std)
            
            # if intersect has more than 1 point, choose the point closest to the middle of the two means
            if len(intersect_i) == 2:
                mean_middle = (light_mean + dark_mean)/2
                intersect = min(intersect_i, key=lambda x:abs(x-mean_middle))
            # if 1 point thats great!
            elif len(intersect_i) == 1:
                intersect = intersect_i
            # if 0 or 3 or more points, simply choose middle of two means
            else:
                intersect = (light_mean + dark_mean)/2
        except:
            # just use middle of means
            if np.isnan(light_mean) or np.isnan(dark_mean):
                intersect = 0
            else:
                intersect = (light_mean + dark_mean)/2
        
        # only add to timestack and data if new data
        if (full_height - snap_height + j) >= snap_end:
            # annotate over timestack
            cv2.circle(timestack_annotated, (min_uprush + int(intersect), full_height - snap_height + j), 1, (255, 0, 0))

            # save vertical and horizontal coordinates for comparison with labels.
            horizontal_coordinates.append(full_height - snap_height + j)
            vertical_coordinates.append(min_uprush + int(intersect))
        
    # get csv values from labelled timestack
    manual_csv_points_filename = folder_name + "runup_data_test" + timestack_name + ".csv"
    x_vals = []
    t_vals = []
    with open(manual_csv_points_filename, newline='') as csvfile:
        points_reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        initial_row = 5
        initial_row_counter = 0
        for row in points_reader:
            if initial_row_counter > initial_row:
                rows = row[0].split(',')
                x_vals.append( int(rows[0][1:-1]) ) # need to remove [] from value and turn into a number
                t_vals.append( int(rows[1][1:-1]) ) # need to remove [] from value and turn into a number
            initial_row_counter = initial_row_counter + 1

    print("red is hand picked, green is found by model")
    plt.plot(x_vals, t_vals, color = 'r')
    plt.plot(vertical_coordinates, horizontal_coordinates, color = 'g')
    plt.gca().invert_yaxis() # invert axis so it is like the timestack image coordinates
    plt.show()

    #full_stack_window = "timestack_annotated"
    #cv2.namedWindow(full_stack_window, cv2.WINDOW_NORMAL)
    #cv2.resizeWindow(full_stack_window, 1080, 1080)
    #cv2.imshow(full_stack_window, timestack_annotated)
    #cv2.waitKey()
    #cv2.destroyAllWindows()

    # find MSE for evey row
    SE = []
    for i in range(0, len(t_vals), 1):
        # make sure the comparison values line up correctly
        index_matching = np.where(np.array(horizontal_coordinates) == t_vals[i])
        vertical_coordinate_matching = vertical_coordinates[ index_matching[0][0] ]
        # /image range normalises the pixel values between 0 and 1
        SE.append( ((x_vals[i] - vertical_coordinate_matching)/image_range)**2 )
    MSE = np.mean(SE)

    # save the image
    print("saving image")
    cv2.imwrite(results_dir + "/" + "annotated_runup_timestack_" + timestack_name + "_" + tf_model_version + ".png", timestack_annotated)
    
    # save the image
    print("saving mask")
    cv2.imwrite(results_dir + "/" + "mask_output_timestack_" + timestack_name + "_" + tf_model_version + ".png", mask_annotated)
    
    # save the data
    print("Saving Data to csv file")

    # create csv file
    output_data_name = results_dir + "/" + "runup_data_" + timestack_name + "_" + tf_model_version + ".csv"
    with open(output_data_name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows([["timestack shoreline coordinates"]])

    with open(output_data_name, 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)

        time_row = list(["time (horizontal coordinate)"])
        time_row.extend(horizontal_coordinates)
        x_row = list(["x (vertical coordinate)"])
        x_row.extend(vertical_coordinates)
        writer.writerow(time_row)
        writer.writerow(x_row)

    csvFile.close()

    return MSE

# plot graphs inside the notebook
%matplotlib inline 

# make a new directory for results
current_path = os.getcwd()
results_dir = tf_model_version
results_dir_path = os.path.join(current_path, results_dir)
if os.path.exists(results_dir_path):
    shutil.rmtree(results_dir_path)
    print("old results directory removed")
os.mkdir(results_dir_path)
print("new results directory created called " + results_dir)

# don't change the min_uprush and max_uprush values, 
# these are fixed values according to the labelled evaluation timestacks
error_list = []
timestack_name = "20101109084810_02_25ppm_test_s.png" # 194 and 925
min_uprush = 194
max_uprush = 925
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101110094203_05_25ppm_test_s.png" # 40 and 580
min_uprush = 40
max_uprush = 580
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "20101111092821_01_25ppm_test_s.png" # 116 and 800
min_uprush = 116
max_uprush = 800
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "OneMile8_test_s.png" # 77 and 289
min_uprush = 77
max_uprush = 289
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

timestack_name = "FC4_test_s.png" # 37 and 240
min_uprush = 37
max_uprush = 240
ground_truth_error = compare_shoreline(min_uprush, max_uprush, tf_model_version, IMG_SIZE, IMG_SIZE_WID, timestack_name, results_dir)
print("MSE for " + timestack_name + " is " + str(ground_truth_error))
error_list.append(ground_truth_error)

print("\nMSE summary for all timestacks:")
print("20101109084810_02_25ppm_test_s.png" + " " + str(error_list[0]))
print("20101110094203_05_25ppm_test_s.png" + " " + str(error_list[1]))
print("20101111092821_01_25ppm_test_s.png" + " " + str(error_list[2]))
print("OneMile8_test_s.png               " + " " + str(error_list[3]))
print("FC4_test_s.png                    " + " " + str(error_list[4]))

print("\nAverage MSE for all seen timestacks = " + str(np.mean(error_list)))

print("done")